In [1]:
# !pip install geemap

# Machine Learning with Earth Engine - Accuracy Assessment

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

To assess the accuracy of a classifier, use a `ConfusionMatrix`. The `sample()` method generates two random samples from the input data: one for training and one for validation. The training sample is used to train the classifier. You can get resubstitution accuracy on the training data from `classifier.confusionMatrix()`. To get validation accuracy, classify the validation data. This adds a `classification` property to the validation `FeatureCollection`. Call `errorMatrix()` on the classified `FeatureCollection` to get a confusion matrix representing validation (expected) accuracy.

![](https://i.imgur.com/vROsEiq.png)

## Step-by-step tutorial

### Import libraries

In [7]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

### Create an interactive map

In [9]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Add data to the map

Let's add the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD), which can be used to create training data with class labels. 

![](https://i.imgur.com/7QoRXxu.png)

In [11]:
NLCD2016 = ee.Image("USGS/NLCD/NLCD2016").select("landcover")
Map.addLayer(NLCD2016, {}, "NLCD 2016")

Load the NLCD metadata to find out the Landsat image IDs used to generate the land cover data.

In [16]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, "NLCD Metadata")

In [18]:
# point = ee.Geometry.Point([-122.4439, 37.7538])  # Sanfrancisco, CA
# point = ee.Geometry.Point([-83.9293, 36.0526])   # Knoxville, TN
point = ee.Geometry.Point([-88.3070, 41.7471])  # Chicago, IL

In [20]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [22]:
metadata.get("2016on_bas").getInfo()

'LC08_2016256'

In [24]:
doy = metadata.get("2016on_bas").getInfo().replace("LC08_", "")
doy

'2016256'

In [25]:
ee.Date.parse("YYYYDDD", doy).format("YYYY-MM-dd").getInfo()

'2016-09-12'

In [27]:
start_date = ee.Date.parse("YYYYDDD", doy)
end_date = start_date.advance(1, "day")

# الحصول علي صورة نفس نتيجة الكود التالي مع اختلاف في صياغة الكود

In [71]:
# # الحصول على صورة LANDSAT
# image = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
#           .filterBounds(point)
#           .filterDate(start_date, end_date)
#           .first()
#           .clip(region) 
#         )

# # تطبيق عوامل القياس على الصورة
# def apply_scale_factors(image):
#   # تحديد النطاقات الضوئية وتطبيق معامل القياس
#   optical_bands = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2)
#   # تحديد النطاقات الحرارية وتطبيق معامل القياس
#   thermal_bands = image.select(['ST_B10']).multiply(0.00341802).add(149.0)
#   # إعادة دمج النطاقات الضوئية والحرارية مع الصورة الأصلية
#   return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# # تطبيق الدالة على الصورة
# image = apply_scale_factors(image)

# # إعدادات العرض المرئي
# visualization = {
#     'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
#     'min': 0.0,
#     'max': 0.3,
# }

# # ضبط المركز والطبقة على الخريطة
# Map.centerObject(point, 8)
# Map.addLayer(image, visualization, 'True Color (432)')
# Map


In [73]:
image = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterBounds(point)
          .filterDate(start_date, end_date)
          .first()
          .clip(region) 
        )
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )
image = apply_scale_factors(image)
# image = image.map(apply_scale_factors)
visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.3,}
Map.centerObject(point, 8)
Map.add_layer(image, visualization, 'True Color (432)')
Map

Map(bottom=24714.0, center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transpa…

In [75]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, "NLCD")

### Prepare for consecutive class labels

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training.

First, we need to use the `remap()` function to turn class labels into consecutive integers.

In [78]:
raw_class_values = nlcd_raw.get("landcover_class_values").getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [80]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [82]:
class_palette = nlcd_raw.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [84]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(
    ["remapped"], ["landcover"]
)
nlcd = nlcd.set("landcover_class_values", new_class_values)
nlcd = nlcd.set("landcover_class_palette", class_palette)

In [86]:
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(bottom=24690.0, center=[41.74672584176937, -88.30535888671875], controls=(WidgetControl(options=['position…

### Make training data

In [88]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [89]:
print(points.size().getInfo())

5000


In [90]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-88.56212124700772, 42.210469414463425]}, 'id': '0', 'properties': {'landcover': 17}}


### Split training and testing

In [94]:
# Use these bands for prediction.
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt("random", split))
validation = sample.filter(ee.Filter.gte("random", split))

In [96]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'SR_B1': 0.0174975,
  'SR_B2': 0.022199999999999998,
  'SR_B3': 0.06402750000000001,
  'SR_B4': 0.039002499999999996,
  'SR_B5': 0.42460749999999997,
  'SR_B6': 0.1770525,
  'SR_B7': 0.07860250000000002,
  'landcover': 17,
  'random': 0.14286352916116452}}

In [97]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '2_0',
 'properties': {'SR_B1': 0.0026199999999999835,
  'SR_B2': 0.011887499999999995,
  'SR_B3': 0.04618,
  'SR_B4': 0.0398,
  'SR_B5': 0.27201000000000003,
  'SR_B6': 0.164595,
  'SR_B7': 0.0903175,
  'landcover': 3,
  'random': 0.7625197716786118}}

### Train the classifier

In this examples, we will use random forest classification.

In [101]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

### Classify the image

In [104]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=24690.0, center=[41.74672584176937, -88.30535888671875], controls=(WidgetControl(options=['position…

### Render categorical map

To render a categorical map, we can set two image properties: `classification_class_values` and `classification_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [107]:
class_values = nlcd.get("landcover_class_values").getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [109]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [110]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [113]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=24690.0, center=[41.74672584176937, -88.30535888671875], controls=(WidgetControl(options=['position…

### Visualize the result

In [115]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [117]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(bottom=24690.0, center=[41.74672584176937, -88.30535888671875], controls=(WidgetControl(options=['position…

### Accuracy assessment

#### Training dataset

`confusionMatrix()` computes a 2D confusion matrix for a classifier based on its training data (ie: resubstitution error). Axis 0 of the matrix correspond to the input classes (i.e., reference data), and axis 1 to the output classes (i.e., classification data). The rows and columns start at class 0 and increase sequentially up to the maximum class value

In [119]:
train_accuracy = classifier.confusionMatrix()

In [121]:
train_accuracy.getInfo()

[[290, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 210, 1, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0],
 [1, 0, 5, 415, 5, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0],
 [0, 0, 0, 15, 187, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0],
 [0, 0, 0, 2, 6, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 213, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 33, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly. Overall accuracy is the easiest to calculate and understand but ultimately only provides the map user and producer with basic accuracy information.

In [123]:
train_accuracy.accuracy().getInfo()

0.9592353106550464

The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

In [127]:
train_accuracy.kappa().getInfo()

0.9421879760316022

Producer's Accuracy is the map accuracy from the point of view of the map maker (the producer). This is how often are real features on the ground correctly shown on the classified map or the probability that a certain land cover of an area on the ground is classified as such. The Producer's Accuracy is complement of the Omission Error, Producer's Accuracy = 100%-Omission Error. It is also the number of reference sites classified accurately divided by the total number of reference sites for that class.

In [131]:
train_accuracy.producersAccuracy().getInfo()

[[0.9965635738831615],
 [0],
 [0.9051724137931034],
 [0.9474885844748858],
 [0.8947368421052632],
 [0.9252336448598131],
 [1],
 [0.9424778761061947],
 [1],
 [0.9333333333333333],
 [0],
 [1],
 [0.8048780487804879],
 [0],
 [0],
 [0],
 [0.8518518518518519],
 [0.9916201117318436],
 [0.8095238095238095],
 [0.7894736842105263]]

The Consumer's Accuracy is the accuracy from the point of view of a map user, not the map maker. the User's accuracy essentially tells use how often the class on the map will actually be present on the ground. This is referred to as reliability. The User's Accuracy is complement of the Commission Error, User's Accuracy = 100%-Commission Error. The User's Accuracy is calculating by taking the total number of correct classifications for a particular class and dividing it by the row total.

In [134]:
train_accuracy.consumersAccuracy().getInfo()

[[0.9931506849315068,
  0,
  0.9333333333333333,
  0.9325842696629213,
  0.935,
  0.9611650485436893,
  1,
  0.9063829787234042,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0.969415619879847,
  0.9444444444444444,
  1]]

#### Validation dataset

In [137]:
validated = validation.classify(classifier)

In [139]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '2_0',
 'properties': {'SR_B1': 0.0026199999999999835,
  'SR_B2': 0.011887499999999995,
  'SR_B3': 0.04618,
  'SR_B4': 0.0398,
  'SR_B5': 0.27201000000000003,
  'SR_B6': 0.164595,
  'SR_B7': 0.0903175,
  'classification': 3,
  'landcover': 3,
  'random': 0.7625197716786118}}

`errorMatrix` computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.

In [145]:
test_accuracy = validated.errorMatrix("landcover", "classification")

In [147]:
test_accuracy.getInfo()

[[128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 18, 13, 1, 2, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 2, 35, 3, 0],
 [3, 0, 12, 100, 27, 1, 0, 6, 0, 0, 0, 0, 1, 0, 0, 0, 4, 40, 0, 0],
 [0, 0, 1, 21, 25, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0],
 [0, 0, 1, 2, 14, 19, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 2, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 4, 10, 0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 10, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [148]:
test_accuracy.accuracy().getInfo()

0.6971586971586972

In [150]:
test_accuracy.kappa().getInfo()

0.5536403674556288

In [151]:
test_accuracy.producersAccuracy().getInfo()

[[0.9922480620155039],
 [0],
 [0.20454545454545456],
 [0.5154639175257731],
 [0.423728813559322],
 [0.475],
 [0],
 [0.5057471264367817],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.1],
 [0.9162087912087912],
 [0],
 [0]]

In [152]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9624060150375939,
  0,
  0.22784810126582278,
  0.5434782608695652,
  0.3424657534246575,
  0.6333333333333333,
  0,
  0.4782608695652174,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.2777777777777778,
  0.8164014687882497,
  0,
  0]]

### Download confusion matrix

In [155]:
import csv
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
training_csv = os.path.join(out_dir, "train_accuracy.csv")
testing_csv = os.path.join(out_dir, "test_accuracy.csv")

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

### Reclassify land cover map

In [158]:
landcover = landcover.remap(new_class_values, raw_class_values).select(
    ["remapped"], ["classification"]
)

In [159]:
landcover = landcover.set("classification_class_values", raw_class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [161]:
Map.addLayer(landcover, {}, "Final land cover")
Map

Map(bottom=24690.0, center=[41.74672584176937, -88.30535888671875], controls=(WidgetControl(options=['position…

### Export the result

Export the result directly to your computer:

In [167]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [169]:
geemap.ee_export_image(landcover, filename=out_file, scale=60)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Lenovo\Downloads\landcover.tif


Export the result to Google Drive:

In [171]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=10
)